In [3]:
import pandas as pd
import re
import pymorphy2

In [4]:
# data and functions declared with bot launch
def get_words_index(df):
    '''
    Returns dict in which keys are terms and values are indexes of questions that contain this term
    '''
    term_dict = set(' '.join(df['вопрос'].values).split())
    rows = df['вопрос'].apply(lambda x: set(x.split()))
    indexes = dict()
    for term in term_dict:
        term_index = []
        for index, row in enumerate(rows):
            if term in row:
                term_index.append(index)
        indexes[term] = term_index
    return indexes

morph = pymorphy2.MorphAnalyzer()
df = pd.read_excel('answers_clear.xls').drop('Unnamed: 0', axis=1)
indexes = get_words_index(df)

In [52]:
def get_answer(message, df=df, indexes=indexes):
    russian_terms =  re.findall(r'[а-яА-Яё]+', message)
    normalized_terms = list(map(lambda x: morph.parse(x)[0].normal_form, russian_terms))
    found_terms = [term for term in normalized_terms if term in indexes]
    terms_index = sorted(list(map(lambda x: set(indexes[x]), found_terms)), key=len, reverse=True)
    result = terms_index[0]
    for index_set in terms_index[1:]:
        intermediate_res = result & index_set
        if intermediate_res:
            result = intermediate_res
        else:
            for ans in df.iloc[list(result), 2:].values:
                yield ans
    if intermediate_res:
        for ans in df.iloc[list(result), 2:].values:
            yield ans

In [65]:
ans = get_answer('зачем нужен разовый пропуск')

In [66]:
next(ans)

array(['Оформить разовый пропуск можно предварительно заполнив форму',
       'https://docs.google.com/forms/d/e/1FAIpQLSdrcGv0ujVO9cHxP9zFggi9wSlLukUmM3G9PmSCxJyYam-ynA/viewform',
       'форму'], dtype=object)

In [59]:
print(get_answer('где написать завяление о восстановлении'))

<generator object get_answer at 0x0000019475323C80>


In [67]:
print(get_answer('зачем нужен разовый пропуск'))

<generator object get_answer at 0x000001947531B2E0>


In [68]:
print(get_answer('как оформить последипломные каникулы'))

<generator object get_answer at 0x0000019474ACA040>
